In [1]:
import pandas as pd
!pip install fasttext
import fasttext
!pip install transformers torch scikit-learn
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
import torch
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
!pip install scikit-learn
from sklearn.linear_model import LogisticRegression

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=aa9ac9afe13be1df0e92d7a93b52bc6e6aa14248e9d793113023f6bd47b50dd7
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.0 MB/s eta 0:00:00
 

In [2]:
dataset=pd.read_csv('/content/Twitter_Data.csv')
dataset.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
dataset = dataset.dropna()
dataset.shape

(162969, 2)

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('stopwords')

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

stop_words = set(stopwords.words('english'))

def clean_remove_stopwords_hashtags_and_tokenize(text):
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s\d.]', '', text)
    text = text.lower()

    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [token for token in tokens if not token.startswith('##')]
    return tokens

dataset['tweets'] = dataset['clean_text'].apply(clean_remove_stopwords_hashtags_and_tokenize)
dataset.rename(columns={'category': 'labels'}, inplace=True)

dataset.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,clean_text,labels,tweets
0,when modi promised “minimum government maximum...,-1.0,"[modi, promised, minimum, government, maximum,..."
1,talk all the nonsense and continue all the dra...,0.0,"[talk, nonsense, continue, drama, vote, modi]"
2,what did just say vote for modi welcome bjp t...,1.0,"[say, vote, modi, welcome, bjp, told, rahul, m..."
3,asking his supporters prefix chowkidar their n...,1.0,"[asking, supporters, prefix, chowkidar, names,..."
4,answer who among these the most powerful world...,1.0,"[answer, among, powerful, world, leader, today..."


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = dataset['tweets']
y = dataset['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train_data = pd.DataFrame({'labels': y_train, 'tweets': X_train})
test_data = pd.DataFrame({'labels': y_test, 'tweets': X_test})

train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [6]:
import csv

def convert_csv_to_fasttext(input_csv, output_txt):
    with open(input_csv, 'r', encoding='utf-8') as csv_file, open(output_txt, 'w', encoding='utf-8') as txt_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            label = row['labels']
            text = row['tweets']
            fasttext_line = f'__label__{label} {text}\n'
            txt_file.write(fasttext_line)

convert_csv_to_fasttext('train_data.csv', 'fasttext_train.txt')

def convert_csv_to_fasttext_test(input_csv, output_txt):
    with open(input_csv, 'r', encoding='utf-8') as csv_file, open(output_txt, 'w', encoding='utf-8') as txt_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            text = row['tweets']
            fasttext_line = f'{text}\n'
            txt_file.write(fasttext_line)

convert_csv_to_fasttext_test('test_data.csv', 'fasttext_test.txt')

In [7]:
model = fasttext.train_supervised(input="fasttext_train.txt", epoch=25, lr=0.5)

model.save_model("sentiment_model.bin")

model = fasttext.load_model("sentiment_model.bin")

def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    return text

with open("fasttext_test.txt", "r", encoding="utf-8") as file:
    test_data = [preprocess_text(line.strip()) for line in file]

correct_predictions = 0
total_predictions = len(test_data)

for sample in test_data:
    prediction = model.predict(sample)
    if prediction[0][0] == '__label__0.0':
        correct_predictions += 1

precision = correct_predictions / total_predictions
recall = correct_predictions / total_predictions

print(f'Number of correct predictions: {correct_predictions}')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')

Number of correct predictions: 16885
Precision: 34.54%
Recall: 34.54%


In [8]:
def get_fasttext_sentiment(text):
    prediction = model.predict(text)
    return prediction[0][0].replace('__label__', '')

Integrated Approach

In [9]:
import torch
import fasttext
from torch.utils.data import Dataset, DataLoader

In [10]:
td=pd.read_csv('train_data.csv')
td.head()

,labels,tweets
0,0.0,"['modi', 'crony', 'capitalist', 'friends', 'gi..."
1,0.0,"['dont', 'data', 'back', 'question', 'narendra..."
2,-1.0,"['nirav', 'modi', 'made', 'death', 'threats', ..."
3,-1.0,"['modi', 'promised', 'orop', 'delivered', 'mod..."
4,0.0,"['ist', 'pappu', 'give', 'bribe', 'spread', 'r..."


In [11]:
import csv
with open("train_data.csv", "r") as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        print(row)

Streaming output truncated to the last 5000 lines.
['-1.0', "['even', 'officers', 'department', 'clueless', 'gst', 'implemented', 'businesses', 'going', 'people', 'pissed', 'wonder', 'modi', 'even', 'mentioning', 'campaigns', 'masterstroke']"]
['1.0', "['one', 'amit', 'shah', 'narendra', 'modi', 'nitin', 'gadkari', 'win', 'highest', 'margin']"]
['0.0', "['people', 'want', 'listen', 'honorable', 'employment', 'development', 'ache', 'din', 'get', 'congress', 'didnt', 'anything', 'pak', 'want', 'defeat', 'modi', 'aree', 'sir', 'indian', 'voting', 'fool', 'know', 'vote']"]
['-1.0', "['code', 'conduct', 'forgotten', 'election', 'commission', 'india', 'ignored', 'due', 'modi']"]
['0.0', "['modis', 'male', 'portraits', 'cellist', 'man', 'glass', 'wine']"]
['-1.0', "['sides', 'equally', 'bad', 'attacking', 'modi', 'everything', 'equals', 'blindly', 'trusting', 'every', 'move', 'defending', 'time']"]
['0.0', "['also', 'chokidar', 'jai', 'hind', 'modi', 'sarkar']"]
['1.0', "['modi', 'get', 'croo

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['1.0', "['nehru', 'wankers', 'stop', 'worrying', 'success', 'wont', 'help', 'modi', 'win', 'elections']"]
['1.0', "['call', 'nation', 'reflected', 'opposition', 'voice', 'bjp', 'modi', 'jumlae', 'pak', 'biryani', 'black', 'money', 'noteban', 'full', 'loot', 'adani', 'ambani', 'reason', 'nation', 'longer', 'beleive', 'anything', 'say']"]
['1.0', "['narendra', 'modi', 'govt', 'pmjdy', 'biggest', 'financial', 'inclusion', 'initiative', 'providing', 'benefits', 'many', 'people', 'rural', 'areas', 'via', 'mynt']"]
['1.0', "['amazing', 'sir', 'potential', 'highly', 'credited', 'raced', 'anyone', 'convinctions', 'indians', 'made', 'mind', 'apna', 'modi', 'aayega', 'phirse', 'baar', 'modi', 'sarkaar']"]
['0.0', "['years', 'sufficient', 'workaholic', 'coward', 'find', 'stashed', 'gandhi', 'public', 'simple', 'modi', 'coward', 'blood', 'hands', 'let', 'police', 'investigators', 'lawyers', 'position', 'repaying', 'paying', 'gandhi']"]
['-1.0', "

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['0.0', "['thr', 'abs', 'prob', 'cong', 'announce', 'never', 'fulfill', 'promise', 'modi', 'wins', 'wont', 'modi', 'loses', 'coalition', 'govt', 'allow', 'raga', 'wont', 'scenario', 'promise', 'liability', 'raga']"]
['0.0', "['modi', 'biopic', 'row', 'issues', 'notice', 'pil', 'seeking', 'stay', 'film']"]
['0.0', "['modi', 'namo', 'namo']"]
['1.0', "['making', 'fool', 'usual', 'sure', 'quoting', 'read', 'whole', 'article', 'try', 'find', 'word', 'demonstrable', 'headline', 'altered', 'modis', 'address', 'allow', 'people', 'like', 'carry', 'misinformation', 'campaign']"]
['0.0', "['union', 'minister', 'vijay', 'goel', 'starts', 'youth', 'modi', 'campaign', 'interaction', 'college', 'students', 'youth', 'various', 'initiatives', 'taken', 'government', 'strengthen', 'yuva', 'shakti', 'country']"]
['0.0', "['simply', 'political', 'vendetta', 'people', 'charged', 'corruption', 'jaitley', 'ddca', 'scam', 'sushma', 'modi', 'gate', 'scandal', 

In [12]:
import csv

train_data = []

with open("train_data.csv", "r") as csv_file:
    reader = csv.reader(csv_file)
    next(reader)
    for row in reader:
        label = float(row[0])
        tweet = eval(row[1])
        train_data.append([label, tweet])

In [13]:
import fasttext

fasttext_model = fasttext.load_model("sentiment_model.bin")

In [14]:
tweet_embeddings = []
for _, tweet in train_data:
    tweet_embedding = fasttext_model.get_word_vector(' '.join(tweet))
    tweet_embeddings.append(tweet_embedding)

In [15]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-dot-v5")
dmodel = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-dot-v5")

In [16]:
print(train_data[11])

[1.0, ['elect', 'member', 'parliament', 'please', 'election', 'election', 'prime', 'minister', 'elected', 'mps', 'elect', 'leader', 'become', 'prime', 'minister', 'india', 'elect', 'modi', 'rahul', 'gandhi', 'anyone', 'else', 'prime', 'minister', 'wake', 'please']]


In [17]:
import torch

tweet_representations = []

for _, tweet in train_data:
  inputs = tokenizer(' '.join(tweet), return_tensors="pt", padding=True, truncation=True)
  with torch.no_grad():
    outputs = dmodel(**inputs)
  tweet_representation = outputs.last_hidden_state.mean(dim=1)
  tweet_representations.append(tweet_representation)

In [18]:
import torch
import numpy as np

combined_embeddings = []

for ft_embedding, db_embedding in zip(tweet_embeddings, tweet_representations):
    ft_embedding = ft_embedding[:db_embedding.shape[-1]]

    ft_embedding = torch.from_numpy(ft_embedding).unsqueeze(0)

    combined_embedding = torch.cat((ft_embedding, db_embedding), dim=1)
    combined_embeddings.append(combined_embedding)

In [23]:
from sklearn.linear_model import LogisticRegression
import torch.nn.functional as F

X_train = [embedding.numpy().flatten() for embedding in combined_embeddings]

X_train = np.array(X_train)

y_train = np.array(y_train).flatten()

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

classifier = LogisticRegression(max_iter=1000)

classifier = LogisticRegression(solver='saga')

classifier.fit(X_train_scaled, y_train)

y_pred = classifier.predict(X_train_scaled)
accuracy = np.mean(y_pred == y_train)

In [28]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_train, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7014498851662897
